<a href="https://colab.research.google.com/github/AmirMohammadiKarbalaei/neural-network/blob/CNN_optimisazion/CNNMnist_keras_tuner_HPoptimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras_tuner

In [3]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Dropout
from keras.utils import np_utils
import numpy as np
import json

In [4]:
#loading data
(X_train,y_train), (X_test, y_test) = mnist.load_data()
print(type(X_train), len(y_test))
X_train = X_train.astype("float32")
X_train/=255 #data normalization
X_test = X_test.astype("float32")
X_test/=255  #data normalization
img_width = X_train.shape[1]
img_height = X_train.shape[2]

11490434/11490434 [==============================] - 1s 0us/step
<class 'numpy.ndarray'> 10000


In [5]:
##Reshaping input data
img_width = 28
img_height = 28
X_train = X_train.reshape(X_train.shape[0],img_width, img_height,1) 
X_test = X_test.reshape(X_test.shape[0],img_width, img_height,1) 

In [6]:
#One hot encoding
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

labels = range(10)
num_classes = y_test.shape[1]

In [7]:
def model_builder(hp):
    
    
    hp_layer_1 = hp.Int('layer_2', min_value=32, max_value=256, step=32)
    hp_layer_2 = hp.Int('layer_1', min_value=32, max_value=256, step=32)
    #hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    #hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    
    model = tf.keras.Sequential()
    model.add(Conv2D(hp_layer_1,
                    (4,4),
                    input_shape = (28,28,1),
                    activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(hp_layer_2,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation = "softmax"))
    model.compile(loss = "categorical_crossentropy", 
                  optimizer = "adam",
                  metrics = ["accuracy"])
    return model

In [8]:
import keras_tuner as kt
import tensorflow as tf
tuner = kt.Hyperband(model_builder,
                    objective = 'val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='dir',
                    project_name = 'x')

In [9]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience=3)

In [10]:
tuner.search(X_train,y_train, epochs=50, validation_split=0.2,callbacks = [stop_early])

Trial 30 Complete [00h 01m 24s]
val_accuracy: 0.9893333315849304

Best val_accuracy So Far: 0.9898333549499512
Total elapsed time: 00h 18m 37s


In [11]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [12]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train,y_train,epochs = 50,validation_split = 0.2,
                   callbacks = [stop_early])

Epoch 1/50
1500/1500 [==============================] - 9s 5ms/step - loss: 0.1801 - accuracy: 0.9452 - val_loss: 0.0569 - val_accuracy: 0.9828
Epoch 2/50
1500/1500 [==============================] - 7s 4ms/step - loss: 0.0689 - accuracy: 0.9783 - val_loss: 0.0522 - val_accuracy: 0.9850
Epoch 3/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0507 - accuracy: 0.9844 - val_loss: 0.0462 - val_accuracy: 0.9868
Epoch 4/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.0391 - accuracy: 0.9877 - val_loss: 0.0461 - val_accuracy: 0.9865
Epoch 5/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.0311 - accuracy: 0.9897 - val_loss: 0.0487 - val_accuracy: 0.9868
Epoch 6/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0260 - accuracy: 0.9912 - val_loss: 0.0523 - val_accuracy: 0.9864
Epoch 7/50
1500/1500 [==============================] - 7s 4ms/step - loss: 0.0231 - accuracy: 0.9920 - val_loss: 0.0434 - val_accuracy:

In [14]:
import pandas as pd
pd.DataFrame(history.history)

,loss,accuracy,val_loss,val_accuracy
0,0.180143,0.945167,0.056899,0.982833
1,0.068874,0.978250,0.052209,0.985000
2,0.050687,0.984417,0.046197,0.986750
3,0.039142,0.987688,0.046130,0.986500
4,0.031117,0.989667,0.048741,0.986750
5,0.025976,0.991229,0.052260,0.986417
6,0.023140,0.992021,0.043376,0.988750
7,0.018729,0.993563,0.057203,0.987667
8,0.016531,0.994271,0.063198,0.988250
9,0.015814,0.994417,0.046884,0.989333


In [15]:
tf.keras.models.save_model(model, "model-CNN-Mnist-besthps.h5")